# Sprawozdanie 6 - akwizycja danych

## Środowisko

Mamy maszynę wirtualną z Ubuntu postawioną za pomocą Vagrant'a (korzystającego pod spodem z VirtualBox'a). Na tej maszynie wirtualnej stawiamy kontenery Docker'a.

Aby ułatwić sobie późniejszą pracę z obrazami na których postawiony jest hadoop postanowiliśmy dodać do master-node volumen na dane (modyfikując skrypty generujące docker-compose). Dzięki temu możemy w wygodny sposób (tj. poprzez wrzucenie do odpowiedniego folderu) przenosić pliki do miejsca, do którego możemy się dostać z poziomu maszyny z hadoopem. Warto zwrócić uwagę, że maszyna wirtualna także posiada taki wolumen, który zapewnia wykorzystanie Vagrant'a.

```yaml
master:
    image: hjben/hadoop-eco:$hadoop_version
    hostname: master
    container_name: master
    privileged: true
    ports:
      - 8088:8088
      - 9870:9870
      - 8042:8042
      - 10000:10000
      - 10002:10002
      - 16010:16010
    volumes:
      - /sys/fs/cgroup:/sys/fs/cgroup
      - $hdfs_path:/data/hadoop
      - $hadoop_log_path:/usr/local/hadoop/logs
      - $hbase_log_path/master:/usr/local/hbase/logs
      - $hive_log_path:/usr/local/hive/logs
      - $sqoop_log_path:/usr/local/sqoop/logs
      - /vagrant/master_volume:/data/master_volume <-------------- dodany volumen
    networks:
      hadoop-cluster:
        ipv4_address: 10.1.2.3
    extra_hosts:
      - "mariadb:10.1.2.2"
      - "master:10.1.2.3"
```

## Pobieranie danych
Niestety przez potrzebę generowania i podania klucza API do serwisu kaggle przed pobraniem danych należy wykonać kilka czynności.

1. Pobrać ze strony kaggle klucz API (kaggle.json)
2. Stworzyć folder .kaggle w głównym katalogu użytkownika i skopiować tam klucz API
3. Wywołać nasz skrypt setup.sh, który:
    1. Przygotowuje środowisko pythonowe
    2. Pobiera z kaggle:
       * [YouTube Trending Video Dataset](https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset)
       * [Steam Dataset](https://www.kaggle.com/datasets/souyama/steam-dataset)
    3. Pobiera z sieci [dane Covid'owe](https://covid.ourworldindata.org/data/owid-covid-data.csv)

In [3]:
import os
from timeit import default_timer as timer
import requests
import docker
import json
import opendatasets as od

In [13]:
if os.getcwd().startswith("/tmp"):
    os.chdir("/vagrant/sprawozdania/akwizycja")

W tym momencie należy się upewnić, że pobrany z kaggle klucz API (kaggle.json) znajduje się w katalogu: `~/.kaggle/kaggle.json`.

In [14]:
output_dir = "../../master_volume/datasets"

In [26]:
%%timeit -r 1 -n 1
if not os.path.isdir(f"{output_dir}/steam-dataset"):
    od.download("https://www.kaggle.com/datasets/souyama/steam-dataset", f"{output_dir}")
else:
    print("Dataset steam-dataset already exists, skipping download")

if not os.path.isdir(f"{output_dir}/youtube-trending-video-dataset"):
    od.download("https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset", f"{output_dir}")
else:
    print("Dataset youtube-trending-video-dataset already exists, skipping download")

Dataset steam-dataset already exists, skipping download
Dataset youtube-trending-video-dataset already exists, skipping download
1.64 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [29]:
%%timeit -r 1 -n 1
path = f"{output_dir}/covid-dataset.csv"

if not os.path.isfile(path):
    print(f"Downloading covid-dataset to {path}")
    start = timer()
    r = requests.get("https://covid.ourworldindata.org/data/owid-covid-data.csv", allow_redirects=True)
    with open(path, 'wb') as file:
        file.write(r.content)
    end = timer()
    print(f"Download finished in {end - start:.02f}s")
else:
    print("Dataset covid-dataset already exists, skipping download")

print(f"covid-dataset.csv: {os.stat(path).st_size / (1024 * 1024):.02f}MB")

Dataset covid-dataset already exists, skipping download
covid-dataset.csv: 77.76MB
973 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Formatowanie danych
Po rozpakowaniu danych widać, że część z nich ma format trudny do późniejszej pracy. Ostatecznie postanowiliśmy przed wrzuceniem plików do hdfs wszystkie przetransformować do dormatu .jsonl. Format .jsonl zawiera obiekty json, każdy w kolejnej linii. Dzięki zastosowaniu takiego formatu na kolejnych laboratoriach będzie można wykorzystywać odpowiednie mappery.


In [16]:
%%timeit -r 1 -n 1

print("Converting JSON to JSONL")
for root, directories, files in os.walk(f"{output_dir}"):
        for filename in files:
            path = os.path.join(root,filename)
            output_path = f"{path}l"

            if os.path.isfile(output_path):
                continue

            if path.endswith(".json"):
                print(path)
                with open(path, "r") as file:
                    data = json.load(file)
                    if type(data) is dict:
                        data = [{"key": key, "value": data[key]} for key in data]

                    with open(output_path, "w") as jsonl_file:
                        for obj in data:
                            json.dump(obj, jsonl_file)
                            jsonl_file.write("\n")
                    print(output_path)

print("Done")

Converting JSON to JSONL
../../master_volume/datasets/steam-dataset/appinfo/dlc_data/missing.json
../../master_volume/datasets/steam-dataset/appinfo/dlc_data/missing.jsonl
../../master_volume/datasets/steam-dataset/appinfo/dlc_data/steam_dlc_data.json
../../master_volume/datasets/steam-dataset/appinfo/dlc_data/steam_dlc_data.jsonl
../../master_volume/datasets/steam-dataset/appinfo/store_data/steam_store_data.json
../../master_volume/datasets/steam-dataset/appinfo/store_data/steam_store_data.jsonl
../../master_volume/datasets/steam-dataset/news_data/missing.json
../../master_volume/datasets/steam-dataset/news_data/missing.jsonl
../../master_volume/datasets/steam-dataset/news_data/steam_news_data.json
../../master_volume/datasets/steam-dataset/news_data/steam_news_data.jsonl
../../master_volume/datasets/steam-dataset/steamspy/basic/steam_spy_scrap.json
../../master_volume/datasets/steam-dataset/steamspy/basic/steam_spy_scrap.jsonl
../../master_volume/datasets/steam-dataset/steamspy/detai

## Dodanie plików do hdfs

In [19]:
client = docker.from_env()
container = client.containers.get('master')

def hdfs_mkdir(path):
    container.exec_run(f"hdfs dfs -mkdir -p /{path}/")

def hdfs_upload(path):
    directory = "/".join(path.split("/")[:-1])
    hdfs_mkdir(directory)
    cmd = f"hdfs dfs -put /data/master_volume/{path} /{directory}"
    print(cmd)
    code, output = container.exec_run(cmd)
    print(f"exit code {code}")
    print(output)

In [18]:
%%timeit -r 1 -n 1

print("Uploading to HDFS")

for root, directories, files in os.walk(f"{output_dir}"):
        for filename in files:
            if filename.endswith(".json"):
                continue # skip JSON files, we have JSONL from previous step
            path = os.path\
                .join(root,filename)\
                .replace("../../master_volume/", "")\
                .replace("\\", "/")
            start_single = timer()
            hdfs_upload(path)
            end_single = timer()
            print(f"HDFS upload took {end_single - start_single:.02f}s")

Uploading to HDFS
hdfs dfs -put /data/master_volume/datasets/steam-dataset/appinfo/dlc_data/missing.jsonl /datasets/steam-dataset/appinfo/dlc_data
exit code 0
b''
hdfs dfs -put /data/master_volume/datasets/steam-dataset/appinfo/dlc_data/steam_dlc_data.jsonl /datasets/steam-dataset/appinfo/dlc_data
exit code 0
b''
hdfs dfs -put /data/master_volume/datasets/steam-dataset/appinfo/dlc_data/timestamp.txt /datasets/steam-dataset/appinfo/dlc_data
exit code 0
b''
hdfs dfs -put /data/master_volume/datasets/steam-dataset/appinfo/store_data/steam_store_data.jsonl /datasets/steam-dataset/appinfo/store_data
exit code 0
b''
hdfs dfs -put /data/master_volume/datasets/steam-dataset/appinfo/store_data/timestamp.txt /datasets/steam-dataset/appinfo/store_data
exit code 0
b''
hdfs dfs -put /data/master_volume/datasets/steam-dataset/news_data/missing.jsonl /datasets/steam-dataset/news_data
exit code 0
b''
hdfs dfs -put /data/master_volume/datasets/steam-dataset/news_data/steam_news_data.jsonl /datasets/ste

In [56]:
def hdfs_set_replication_level(number):
    container.exec_run(f"hdfs dfs -setrep -R {number} /")

hdfs_set_replication_level(3)

## Akwizycja danych zmiennych

Nasz proces zakłada, że na podstawie części danych niezmiennych dotyczących gier z serwisu Steam (steam_dataset) wytypujemy te gry, o których liczby graczy na przestrzeni czasu będziemy pytać SteamCharts API za pomocą bardzo prostego zapytania
```
https://steamcharts.com/app/<appid>/chart-data.json
```

W celu akwizycji tych danych przygotowaliśmy proces map-reduce, który przyjmuje na wejściu potrzebne id i zwraca wyniki zapytania w odpowiedniej postaci. Na potrzeby tej listy zadań umieściliśmy w hdfs odpowiedni plik z wybranymi przez nas identyfikatorami do celów testowych. W przyszłości ten krok będzie wykorzystywał wyniki z poprzednich podprocesów.